## 1. Tensorflow 基础数据类型

- 基础数据类型
    1. tf.constant
    2. tf.string
    3. tf.ragged.constant, tf.SpareTensor, tf.Variable
- 自定义损失函数
    1. tf.reduce_mean
- 自定义层次
    1. keras.layers.lambda
    2. 继承法
- TF.function
    1. tf.function
    2. tf.autograph.to_code
    3. get_concrete_function
- GraphDef
    1. get_operations
    2. get_operation_by_name
    3. get_tensor_by_name
    4. as_graph_def
- 自动求导
    1. tf.GradientTape
    2. optimzier.apply_gradients

In [73]:
import matplotlib as mpl
import matplotlib.pyplot as plt
% matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

UsageError: Line magic function `%` not found.


### 1. tf.constant

#### 1.1 常量的定义以及索引操作

In [74]:
# 定义常量
t = tf.constant([[1., 2., 3., ], [4., 5., 6.]])

# tensorflow 的索引
print(t)
# 打印 从第二列开始到最后一列的数据
print(t[:, 1:])
# 获取第二列的数据
print(t[..., 1])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


#### 1.2  tensorflow 的基本操作

In [75]:
# tensorflow 的基本操作
print(t + 10)  # tensor 中的每个数据都加10
print(tf.square(t))  # 对t中的每个数据求平方
print(t @ tf.transpose(t))  # t 和他的转置相乘 (2,3) * (3,2)

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


#### 1.3 tf和numpy的相互转换

In [76]:
print(t.numpy())
print(tf.square(t))

np_t = np.array([[1., 2., 3.], [4., 5., 6.]])
print(tf.constant(np_t))

[[1. 2. 3.]
 [4. 5. 6.]]
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


#### 1.4 标量

In [77]:
# 定义标量(标量一般是0维的)
t = tf.constant(3.2)
print(t)
print(t.numpy())
print(t.shape)

tf.Tensor(3.2, shape=(), dtype=float32)
3.2
()


### 2. tf.string和ragged_tensor

#### 2.1 string存储

In [78]:
t = tf.constant("hello")
print(t)

# 打印字符的长度
print(tf.strings.length(t))
# unit : BYTE(默认)
print(tf.strings.length(t, unit="UTF8_CHAR"))
# 将数据转换成 unicode 数组
print(tf.strings.unicode_decode(t, "UTF8"))

tf.Tensor(b'hello', shape=(), dtype=string)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor([104 101 108 108 111], shape=(5,), dtype=int32)


In [79]:
# string array
t = tf.constant(["caffe", "world", "你好"])
# 返回字符长度
print(tf.strings.length(t, unit="UTF8_CHAR"))
# 返回 unicode 字符数组
u_t = tf.strings.unicode_decode(t, input_encoding="UTF8")
print(u_t)

tf.Tensor([5 5 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 102, 101], [119, 111, 114, 108, 100], [20320, 22909]]>


#### 2.2 非定长数组TaggedTensor

In [80]:
# ragged tensor => 不等长数组
# tf.ragged.constant 创建非定长的数组
r = tf.ragged.constant([[1, 2], [4, 5, 6], [], [7, 8]], dtype=tf.float32)
print(r)
print(r.shape)
# index , 不能将等长的数组的索引方式用在非等长的索引
print(r[1])
print(r[1:3])
print(r[1, 2])

<tf.RaggedTensor [[1.0, 2.0], [4.0, 5.0, 6.0], [], [7.0, 8.0]]>
(4, None)
tf.Tensor([4. 5. 6.], shape=(3,), dtype=float32)
<tf.RaggedTensor [[4.0, 5.0, 6.0], []]>
tf.Tensor(6.0, shape=(), dtype=float32)


#### 2.3 非定长数组的合并

In [81]:
# 非定长数组的合并
r2 = tf.ragged.constant([[1, 3], [4, 5, 6]], dtype=tf.float32)
# 在行上合并r 和 r2, axis = 0 表示x轴(即行)
print(tf.concat([r, r2], axis=0))

<tf.RaggedTensor [[1.0, 2.0], [4.0, 5.0, 6.0], [], [7.0, 8.0], [1.0, 3.0], [4.0, 5.0, 6.0]]>


In [82]:
# 在列上合并
r3 = tf.ragged.constant([[1, 3], [4, 5, 6], [1], [2]], dtype=tf.float32)
# 列合并要求 shape 相同
print(r.shape)
print(r3.shape)
print(tf.concat([r, r3], axis=1))

(4, None)
(4, None)
<tf.RaggedTensor [[1.0, 2.0, 1.0, 3.0], [4.0, 5.0, 6.0, 4.0, 5.0, 6.0], [1.0], [7.0, 8.0, 2.0]]>


#### 2.4 RaggedTensor转Tensor

In [83]:
# 对每行的数据进行对齐, 不足是使用0代替
# 注意 RaggedTensor 会把0值放到非0值的后面
print(r3.to_tensor())

tf.Tensor(
[[1. 3. 0.]
 [4. 5. 6.]
 [1. 0. 0.]
 [2. 0. 0.]], shape=(4, 3), dtype=float32)


### 3. SparseTensor和tf.Variable

- Note :
  1. RaggedTensor 会把0值放到非0值的后面, 有一些情况下我们需要把0值放到非0值前面, 所以不能使用 RaggedTensor
  2. 如果一个矩阵中大部分值都是 0, 少部分是非0值, 我们可以使用 SparseTensor, 只需要记录非0值的index即可
  3. 类似于稀疏矩阵



#### 3.1 SparseTensor的创建

- tf.SparseTensor()
    1. indices : 非0值的位置
    2. values : 实际的非0值
    3. dense_shape: SparseTensor的shape

In [84]:
s = tf.SparseTensor(
    indices=[[0, 1], [1, 0], [2, 3]],  # 非0值的位置
    values=[1, 2, 3],  # 实际的值
    dense_shape=[3, 4]
)
print(s)
# 转成 tensor
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1 2 3], shape=(3,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0 1 0 0]
 [2 0 0 0]
 [0 0 0 3]], shape=(3, 4), dtype=int32)


#### 3.2 SparseTensor基本操作

- Notes :
    1. SparseTensor无法做加法操作
    2. SparseTensor的indices必须是排好序的, 如果没有排序使用 to_dense 报错, 可以使用 tf.sparse.reorder(s) 解决

In [85]:
s1 = tf.SparseTensor(
    indices=[[1, 1], [0, 2], [1, 3]],
    values=[3, 4, 6],
    dense_shape=[3, 4]
)
print(s1)
# 如果 indices 不是排好序的, 那么就需要排序后才能to_dense
s2 = tf.sparse.reorder(s1)
print(tf.sparse.to_dense(s2))

SparseTensor(indices=tf.Tensor(
[[1 1]
 [0 2]
 [1 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([3 4 6], shape=(3,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0 0 4 0]
 [0 3 0 6]
 [0 0 0 0]], shape=(3, 4), dtype=int32)


In [86]:
s3 = s2 * 20
print(s3)

# 加法操作会报错 unsupported operand type(s) for +: 'SparseTensor' and 'int'
try:
    s3 = s2 + 1
except TypeError as ex:
    print(ex)

# 和密集矩阵相乘, 注意dtype要相同
s4 = tf.constant(np.ones(shape=[4, 3]), dtype=tf.int32)
print(tf.sparse.to_dense(s))
print(s4)
print(tf.sparse.sparse_dense_matmul(s, s4))

SparseTensor(indices=tf.Tensor(
[[0 2]
 [1 1]
 [1 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([ 80  60 120], shape=(3,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[0 1 0 0]
 [2 0 0 0]
 [0 0 0 3]], shape=(3, 4), dtype=int32)
tf.Tensor(
[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]], shape=(4, 3), dtype=int32)
tf.Tensor(
[[1 1 1]
 [2 2 2]
 [3 3 3]], shape=(3, 3), dtype=int32)


#### 3.3 Variables(变量)

In [88]:
# 定义Variable
v = tf.Variable([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
print(v)
print(v.value())  # 返回Tensor
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


#### 3.4 修改变量

- 修改变量只能用 v.assign的方式

In [93]:
v.assign(2 * v)
print(v.numpy())
v[0, 1].assign(996)
print(v.numpy())
v[1].assign([6, 6, 6])
print(v.numpy())

[[  32. 1992.   96.]
 [ 128.  160.  192.]]
[[ 32. 996.  96.]
 [128. 160. 192.]]
[[ 32. 996.  96.]
 [  6.   6.   6.]]


In [95]:
# 直接用 = 赋值会报错
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
